In [2]:
!pip install pulp
from pulp import LpMaximize, LpProblem, LpVariable

# Define the LP problem
model = LpProblem("Chemical_Manufacturing_Optimization", LpMaximize)

# Decision variables
P = LpVariable("Primary_Product", lowBound=0, cat="Continuous")
K = LpVariable("Product_K", lowBound=0, cat="Continuous")
M = LpVariable("Product_M", lowBound=0, cat="Continuous")
T = LpVariable("Waste_Treated", lowBound=0, cat="Continuous")

# Objective function: Maximize profit
model += (
    5.70 * P + 0.80 * K + 0.65 * M - 0.25 * T
    - 0.20 * K - 0.10 * M,
    "Total_Profit"
)

# Constraints
model += P + K <= 7500, "Material_X_Limit"
model += 2 * P + M <= 9000, "Material_Y_Limit"
model += P == K + M + T, "Waste_Balance"

# Solve the model
model.solve()

# Print results
print("Optimal Production Plan:")
print(f"Primary Product (P): {P.varValue} lbs")
print(f"Product K (K): {K.varValue} lbs")
print(f"Product M (M): {M.varValue} lbs")
print(f"Waste Treated (T): {T.varValue} lbs")
print(f"Total Profit: ${model.objective.value():,.2f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 84.9 MB/s eta 0:00:00
Optimal Production Plan:
Primary Product (P): 4500.0 lbs
Product K (K): 3000.0 lbs
Product M (M): 0.0 lbs
Waste Treated (T): 1500.0 lbs
Total Profit: $27,075.00


In [4]:
# Sensitivity Analysis: Varying Parameters
price_changes = [5.50, 5.70, 5.90]  # Test different selling prices of primary product
profit_results = {}

for price in price_changes:
    # Rebuild the objective function with the new price
    model.setObjective(
        price * P + 0.80 * K + 0.65 * M - 0.25 * T - 0.20 * K - 0.10 * M
    )
    model.solve()  # Re-solve the model
    profit_results[price] = model.objective.value()

# Print Sensitivity Analysis Results
print("\nSensitivity Analysis on Primary Product Price:")
for price, profit in profit_results.items():
    print(f"Price: ${price:.2f} -> Total Profit: ${profit:,.2f}")

# Shadow Prices (Constraint Sensitivity)
print("\nShadow Prices (Dual Values for Constraints):")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.pi:.2f} (Shadow Price)")

# Reduced Costs (Cost Sensitivity)
print("\nReduced Costs (Impact of Non-Binding Variables):")
for var in [P, K, M, T]:
    print(f"{var.name}: {var.dj:.2f} (Reduced Cost)")


Sensitivity Analysis on Primary Product Price:
Price: $5.50 -> Total Profit: $26,175.00
Price: $5.70 -> Total Profit: $27,075.00
Price: $5.90 -> Total Profit: $27,975.00

Shadow Prices (Dual Values for Constraints):
Material_X_Limit: 0.85 (Shadow Price)
Material_Y_Limit: 2.40 (Shadow Price)
Waste_Balance: 0.25 (Shadow Price)

Reduced Costs (Impact of Non-Binding Variables):
Primary_Product: -0.00 (Reduced Cost)
Product_K: -0.00 (Reduced Cost)
Product_M: -1.60 (Reduced Cost)
Waste_Treated: -0.00 (Reduced Cost)
